In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
from place_groups import place_groups_middle_of_roads, place_groups_junctions

from sklearn.linear_model import LinearRegression
from datetime import timedelta
from keras.callbacks import ModelCheckpoint
import numpy as np
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pandas as pd
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras import regularizers

from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16, 10)

# prevent tensorflow from allocating the entire GPU memory at once
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

Using TensorFlow backend.


In [3]:
HDF5_PREFIX = 'only_speed_lstm_simple'

In [4]:
HDF5_DIR = os.path.join('.', 'hdf5')
ONE_DAY_LAGS = 288
DATA_PATH = '/mnt/sdc1/inon/norrecampus/data/by_place_5min'
EXAMPLE_PLACE_ID = 'ChIJZaR1M1hSUkYRxP0WkwYYy_k'

THRESHOLD_HIGH_OUTLIER = 110
LOSS = 'mse'
MINI_BATCH_SIZE = 512
NUM_EPOCHS = 100
VALIDATION_SPLIT = 0.2
NUM_LAGS = 12
LSTM_STATE_SIZE = NUM_LAGS
NUM_OUTS_MEAN_SPEED_REGRESSION = 1
SPLIT_DATE = '2015-06-01'

In [5]:
def split_timestamp(ser):
    return pd\
        .DataFrame(ser)\
        .assign(day_of_week=lambda df: df.index.dayofweek, time_of_day=lambda df: df.index.time)\
        .reset_index()

In [6]:
class LearnerForMeanSpeedOfOneSegment(object):
    def __init__(self,
                 threshold_high_outlier,
                 loss,
                 place_id, 
                 mini_batch_size,
                 num_epochs,
                 validation_split,
                 num_lags,
                 lstm_state_size,
                 num_outs,
                 split_date):
        self.df = None
        self.model = None
        
        self.threshold_high_outlier = threshold_high_outlier
        self.loss = loss
        self.place_id = place_id
        self.mini_batch_size = mini_batch_size
        self.num_epochs = num_epochs
        self.validation_split = validation_split
        self.num_lags = num_lags
        self.lstm_state_size = lstm_state_size
        self.num_outs = num_outs
        self.split_date = split_date
        self.checkpoint_basename = os.path.join(HDF5_DIR, '%s_%s.best.hdf5' % (HDF5_PREFIX, self.place_id))
    
    def compute_error_statistics(self, errors_df):
        abs_errors = errors_df.error.abs()
        abs_errors_normalized = abs_errors / errors_df.speed_km_hr_true
        return pd.DataFrame({
            'corr': np.corrcoef(errors_df.speed_km_hr_predicted, errors_df.speed_km_hr_true)[0, 1],
            'mae': np.mean(abs_errors),
            'mape': np.mean(abs_errors_normalized),
            'mse': np.mean(abs_errors ** 2),
            'msne': np.mean(abs_errors_normalized ** 2),
            'rae': np.sum(abs_errors) / np.sum(np.abs(errors_df.speed_km_hr_true - np.mean(errors_df.speed_km_hr_true))),
            'rmse': np.sqrt(np.mean(abs_errors ** 2)),
            'rmsne': np.sqrt(np.mean(abs_errors_normalized ** 2)),
            'r2': max(0, 1 - np.sum(abs_errors ** 2) / np.sum((errors_df.speed_km_hr_true - np.mean(errors_df.speed_km_hr_true)) ** 2))
        }, index=[self.place_id])
    
    def baseline_lr(self):
        X_train_normalized, Y_train_normalized, X_test_normalized, Y_test_normalized = self._get_train_and_test_inputs()
        trained_lr = LinearRegression(fit_intercept=False).fit(X_train_normalized.values, Y_train_normalized.values)
        lr_predictions_normalized = trained_lr.predict(X_test_normalized)
        errors_df = self._individual_errors_without_interpolated_values(lr_predictions_normalized)
        return self.compute_error_statistics(errors_df)
        
    def _get_detrend_factors(self):
        ser_imp = self._imputation_for_missing_values_and_outliers(self._get_df()).speed_km_hr
        df_train_period = ser_imp[lambda df: df.index < self.split_date]
        return df_train_period\
            .groupby([df_train_period.index.dayofweek, df_train_period.index.time])\
            .agg(['mean', 'std'])\
            .reset_index()\
            .rename(columns={'level_0': 'day_of_week', 'level_1': 'time_of_day'})
        
    def detrend(self, ser):
        return pd\
            .merge(left=split_timestamp(ser), 
                   right=self._get_detrend_factors(),
                   on=['day_of_week', 'time_of_day'], 
                   how='inner')\
                .assign(speed_normalized=lambda df: (df.speed_km_hr - df['mean']) / df['std'])\
                .set_index('index')\
                .speed_normalized\
                .sort_index()

    def _map_back(self, detrended_series):
        return pd\
            .merge(left=split_timestamp(detrended_series), 
                   right=self._get_detrend_factors(),
                   on=['day_of_week', 'time_of_day'], 
                   how='inner')\
            .assign(speed_km_hr=lambda df: (df.speed_normalized * df['std']) + df['mean'])\
            .set_index('index')\
            .speed_km_hr\
            .sort_index()
    
    def _imputation_for_missing_values_and_outliers(self, ser):
        missing_timestamps = pd\
            .date_range(min(ser.index.date), max(ser.index.date) + timedelta(days=1), freq='5T')\
            .difference(ser.index)
        df_with_nans_where_missing = pd.DataFrame(ser)\
            .join(pd.DataFrame(index=missing_timestamps), how='outer')\
            .assign(original_value=lambda df: df.iloc[:, 0])
        return df_with_nans_where_missing\
            .iloc[:, 0]\
            .mask(lambda df: df > self.threshold_high_outlier)\
            .interpolate()\
            .to_frame()\
            .assign(original_value=df_with_nans_where_missing.original_value)\
            .assign(is_interpolated=lambda df: df.iloc[:, 0] != df.iloc[:, 1])
        
    def _get_df(self):
        if self.df is None:
            self.df = pd.read_csv(
                os.path.join(DATA_PATH, self.place_id + '.csv'),
                parse_dates=['start_interval_s', 'end_interval_s']\
            )[lambda df: df.start_interval_s >= '2015-01-01']\
            .rename(columns={'start_interval_s': 't', 'speed_mean': 'speed_km_hr'})\
            .set_index('t')\
            .speed_km_hr
        return self.df

    def _get_train_and_test_inputs(self):
        speeds_interpolated_and_detrended = self.detrend(
            self._imputation_for_missing_values_and_outliers(self._get_df()).speed_km_hr)
        lags = pd.concat([speeds_interpolated_and_detrended.shift(x) for x in range(self.num_lags + 1)], axis=1)[self.num_lags:]
        train = lags[lags.index < self.split_date]
        X_train = train.iloc[:, 1:]
        Y_train = train.iloc[:, 0]
        test = lags[lags.index >= self.split_date]
        X_test = test.iloc[:, 1:]
        Y_test = test.iloc[:, 0]
        return X_train, Y_train, X_test, Y_test

    def _build_model(self):
        # TODO: better weights initialization
        self.model = Sequential()
        self.model.add(LSTM(self.lstm_state_size, input_shape=(self.num_lags, 1), return_sequences=False))
        # model.add(Dropout(0.2))
        #model.add(Dense(units=1, kernel_regularizer=regularizers.l2(0.00001)))
        self.model.add(Dense(units=self.num_outs, activation="linear"))  # Linear activation, because speed RESIDUALS can have any sign.
        self.model.compile(loss=self.loss, optimizer="rmsprop")  # TODO: try adam optimizer too, although rmsprop is the default go-to for RNN
    
    def load_best_model_from_disk(self):
        self._build_model()
        self.model.load_weights(self.checkpoint_basename)
    
    def create_and_train_model(self):
        self._build_model()
        X_train, Y_train, _, _ = self._get_train_and_test_inputs()
        self.model.fit(
            np.expand_dims(np.array(X_train.values), 2),
            Y_train.values,
            batch_size=self.mini_batch_size,
            epochs=self.num_epochs,
            validation_split=self.validation_split,
            # checkpoint best model
            callbacks=[ModelCheckpoint(
                self.checkpoint_basename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')],
            verbose=2)
        self.model.load_weights(self.checkpoint_basename)
    
    def predict(self):
        _, _, X_test_normalized, Y_test_normalized = self._get_train_and_test_inputs()
        predictions_normalized = self.model.predict(np.expand_dims(X_test_normalized, 2)).flatten()
        errors_df = self._individual_errors_without_interpolated_values(predictions_normalized)
        return self.compute_error_statistics(errors_df)
        
    def _impute(self):
        raise NotImplementedError
        
    def _individual_errors_without_interpolated_values(self, predictions_normalized):
        _, _, _, Y_test_normalized = self._get_train_and_test_inputs()
        ser_Y_test = self._map_back(Y_test_normalized)
        ser_predictions = self._map_back(pd.Series(
            predictions_normalized, index=Y_test_normalized.index, name='speed_normalized'))
        interpolated_timestamps = self\
            ._imputation_for_missing_values_and_outliers(self._get_df())\
            [lambda df: df.is_interpolated]\
            .index
        return ser_Y_test.to_frame()\
            .join(ser_predictions.to_frame(), lsuffix='_true', rsuffix='_predicted')\
            .loc[lambda df: df.index.difference(interpolated_timestamps)]\
            .assign(error=lambda df: df.speed_km_hr_true - df.speed_km_hr_predicted)

In [7]:
def one_place_improvement_over_baseline_lr(place_id):
    learner = LearnerForMeanSpeedOfOneSegment(
        threshold_high_outlier=THRESHOLD_HIGH_OUTLIER,
        loss=LOSS,
        place_id=place_id, 
        mini_batch_size=MINI_BATCH_SIZE,
        num_epochs=NUM_EPOCHS,
        validation_split=VALIDATION_SPLIT,
        num_lags=NUM_LAGS,
        lstm_state_size=LSTM_STATE_SIZE,
        num_outs=NUM_OUTS_MEAN_SPEED_REGRESSION,
        split_date=SPLIT_DATE)
    learner.create_and_train_model()
    learner.load_best_model_from_disk()
    return learner.predict() - learner.baseline_lr()

In [8]:
# one_place_improvement_over_baseline_lr(EXAMPLE_PLACE_ID)

In [9]:
def multiple_places(place_group, place_group_name):
    places = [lst[len(lst) // 2] for lst in place_group]
    return pd.concat(map(one_place_improvement_over_baseline_lr, places))\
        .to_csv('%s_improvement_over_baseline_lr_v1.csv' % place_group_name)

In [ ]:
multiple_places(place_groups_middle_of_roads, 'place_groups_middle_of_roads')

Train on 34780 samples, validate on 8696 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.76010, saving model to ./hdf5/only_speed_lstm_simple_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 2s - loss: 0.9336 - val_loss: 0.7601
Epoch 2/100
Epoch 00002: val_loss improved from 0.76010 to 0.64227, saving model to ./hdf5/only_speed_lstm_simple_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 1s - loss: 0.8083 - val_loss: 0.6423
Epoch 3/100
Epoch 00003: val_loss improved from 0.64227 to 0.53455, saving model to ./hdf5/only_speed_lstm_simple_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 1s - loss: 0.6728 - val_loss: 0.5346
Epoch 4/100
Epoch 00004: val_loss improved from 0.53455 to 0.46829, saving model to ./hdf5/only_speed_lstm_simple_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 1s - loss: 0.5725 - val_loss: 0.4683
Epoch 5/100
Epoch 00005: val_loss improved from 0.46829 to 0.42656, saving model to ./hdf5/only_speed_lstm_simple_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 1s - loss: 0.5126 - val_loss: 0.

Epoch 62/100
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.2801 - val_loss: 0.2448
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.2834 - val_loss: 0.2411
Epoch 64/100
Epoch 00064: val_loss improved from 0.23402 to 0.22951, saving model to ./hdf5/only_speed_lstm_simple_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 1s - loss: 0.2797 - val_loss: 0.2295
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.2800 - val_loss: 0.2310
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.2785 - val_loss: 0.2752
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.2792 - val_loss: 0.2604
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.2772 - val_loss: 0.2466
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.2796 - val_loss: 0.2782
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.2773 - val_loss: 0.2370
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 1s - loss: 0.2770 - val_l

Epoch 26/100
Epoch 00026: val_loss improved from 0.32455 to 0.31945, saving model to ./hdf5/only_speed_lstm_simple_ChIJ3WKEa1dSUkYR4s4cJpaCZfQ.best.hdf5
 - 1s - loss: 0.3626 - val_loss: 0.3194
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 1s - loss: 0.3584 - val_loss: 0.3477
Epoch 28/100
Epoch 00028: val_loss improved from 0.31945 to 0.31452, saving model to ./hdf5/only_speed_lstm_simple_ChIJ3WKEa1dSUkYR4s4cJpaCZfQ.best.hdf5
 - 1s - loss: 0.3548 - val_loss: 0.3145
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 1s - loss: 0.3571 - val_loss: 0.3350
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 1s - loss: 0.3523 - val_loss: 0.3262
Epoch 31/100
Epoch 00031: val_loss improved from 0.31452 to 0.30939, saving model to ./hdf5/only_speed_lstm_simple_ChIJ3WKEa1dSUkYR4s4cJpaCZfQ.best.hdf5
 - 1s - loss: 0.3507 - val_loss: 0.3094
Epoch 32/100
Epoch 00032: val_loss improved from 0.30939 to 0.30896, saving model to ./hdf5/only_speed_lstm_simple_ChIJ3WKEa1dSUkYR4s4cJpaCZfQ.best.

Epoch 00098: val_loss did not improve
 - 1s - loss: 0.2860 - val_loss: 0.2580
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 1s - loss: 0.2877 - val_loss: 0.2666
Epoch 100/100
Epoch 00100: val_loss did not improve
 - 1s - loss: 0.2852 - val_loss: 0.2770
Train on 34780 samples, validate on 8696 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.80755, saving model to ./hdf5/only_speed_lstm_simple_ChIJCy4tOFhSUkYRBiS5iJUrGnY.best.hdf5
 - 2s - loss: 0.9589 - val_loss: 0.8076
Epoch 2/100
Epoch 00002: val_loss improved from 0.80755 to 0.68236, saving model to ./hdf5/only_speed_lstm_simple_ChIJCy4tOFhSUkYRBiS5iJUrGnY.best.hdf5
 - 1s - loss: 0.8442 - val_loss: 0.6824
Epoch 3/100
Epoch 00003: val_loss improved from 0.68236 to 0.57434, saving model to ./hdf5/only_speed_lstm_simple_ChIJCy4tOFhSUkYRBiS5iJUrGnY.best.hdf5
 - 1s - loss: 0.7150 - val_loss: 0.5743
Epoch 4/100
Epoch 00004: val_loss improved from 0.57434 to 0.46224, saving model to ./hdf5/only_speed_lstm_simple_ChIJ

Epoch 58/100
Epoch 00058: val_loss improved from 0.25666 to 0.25270, saving model to ./hdf5/only_speed_lstm_simple_ChIJCy4tOFhSUkYRBiS5iJUrGnY.best.hdf5
 - 1s - loss: 0.3095 - val_loss: 0.2527
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 1s - loss: 0.3093 - val_loss: 0.2888
Epoch 60/100
Epoch 00060: val_loss did not improve
 - 1s - loss: 0.3066 - val_loss: 0.2601
Epoch 61/100
Epoch 00061: val_loss improved from 0.25270 to 0.25090, saving model to ./hdf5/only_speed_lstm_simple_ChIJCy4tOFhSUkYRBiS5iJUrGnY.best.hdf5
 - 1s - loss: 0.3063 - val_loss: 0.2509
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.3057 - val_loss: 0.2697
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.3035 - val_loss: 0.2603
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.3044 - val_loss: 0.2698
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.3014 - val_loss: 0.2735
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.3

...

Epoch 73/100
Epoch 00073: val_loss did not improve
 - 1s - loss: 0.2042 - val_loss: 0.1512
Epoch 74/100
Epoch 00074: val_loss did not improve
 - 1s - loss: 0.2015 - val_loss: 0.2123
Epoch 75/100
Epoch 00075: val_loss did not improve
 - 1s - loss: 0.2023 - val_loss: 0.1998
Epoch 76/100
Epoch 00076: val_loss did not improve
 - 1s - loss: 0.2012 - val_loss: 0.1554
Epoch 77/100
Epoch 00077: val_loss did not improve
 - 1s - loss: 0.2009 - val_loss: 0.1857
Epoch 78/100
Epoch 00078: val_loss did not improve
 - 1s - loss: 0.2009 - val_loss: 0.1867
Epoch 79/100
Epoch 00079: val_loss did not improve
 - 1s - loss: 0.1987 - val_loss: 0.1483
Epoch 80/100
Epoch 00080: val_loss did not improve
 - 1s - loss: 0.1983 - val_loss: 0.1499
Epoch 81/100
Epoch 00081: val_loss did not improve
 - 1s - loss: 0.1968 - val_loss: 0.1726
Epoch 82/100
Epoch 00082: val_loss improved from 0.14736 to 0.13681, saving model to ./hdf5/only_speed_lstm_simple_ChIJC9G6K_5SUkYRWa6ieQhstlo.best.hdf5
 - 1s - loss: 0.1944 - val_l

Epoch 37/100
Epoch 00037: val_loss did not improve
 - 1s - loss: 0.2950 - val_loss: 0.2518
Epoch 38/100
Epoch 00038: val_loss improved from 0.23919 to 0.23906, saving model to ./hdf5/only_speed_lstm_simple_ChIJdbHNKf5SUkYRH2U_kHcp420.best.hdf5
 - 1s - loss: 0.2978 - val_loss: 0.2391
Epoch 39/100
Epoch 00039: val_loss did not improve
 - 2s - loss: 0.2937 - val_loss: 0.2910
Epoch 40/100
Epoch 00040: val_loss did not improve
 - 1s - loss: 0.2909 - val_loss: 0.2427
Epoch 41/100
Epoch 00041: val_loss did not improve
 - 1s - loss: 0.2909 - val_loss: 0.3082
Epoch 42/100
Epoch 00042: val_loss did not improve
 - 2s - loss: 0.2864 - val_loss: 0.2777
Epoch 43/100
Epoch 00043: val_loss did not improve
 - 2s - loss: 0.2893 - val_loss: 0.2530
Epoch 44/100
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.2862 - val_loss: 0.2564
Epoch 45/100
Epoch 00045: val_loss did not improve
 - 2s - loss: 0.2857 - val_loss: 0.2413
Epoch 46/100
Epoch 00046: val_loss improved from 0.23906 to 0.23817, saving mod

Epoch 7/100
Epoch 00007: val_loss improved from 0.39450 to 0.39249, saving model to ./hdf5/only_speed_lstm_simple_ChIJsde_glVSUkYRlK8i5BD3BOA.best.hdf5
 - 1s - loss: 0.4362 - val_loss: 0.3925
Epoch 8/100
Epoch 00008: val_loss improved from 0.39249 to 0.37082, saving model to ./hdf5/only_speed_lstm_simple_ChIJsde_glVSUkYRlK8i5BD3BOA.best.hdf5
 - 1s - loss: 0.4238 - val_loss: 0.3708
Epoch 9/100
Epoch 00009: val_loss improved from 0.37082 to 0.35997, saving model to ./hdf5/only_speed_lstm_simple_ChIJsde_glVSUkYRlK8i5BD3BOA.best.hdf5
 - 1s - loss: 0.4150 - val_loss: 0.3600
Epoch 10/100
Epoch 00010: val_loss did not improve
 - 1s - loss: 0.4110 - val_loss: 0.3655
Epoch 11/100
Epoch 00011: val_loss improved from 0.35997 to 0.35759, saving model to ./hdf5/only_speed_lstm_simple_ChIJsde_glVSUkYRlK8i5BD3BOA.best.hdf5
 - 1s - loss: 0.4046 - val_loss: 0.3576
Epoch 12/100
Epoch 00012: val_loss did not improve
 - 1s - loss: 0.3995 - val_loss: 0.3737
Epoch 13/100
Epoch 00013: val_loss did not improv

Epoch 77/100
Epoch 00077: val_loss did not improve
 - 1s - loss: 0.2676 - val_loss: 0.2583
Epoch 78/100
Epoch 00078: val_loss improved from 0.23163 to 0.22930, saving model to ./hdf5/only_speed_lstm_simple_ChIJsde_glVSUkYRlK8i5BD3BOA.best.hdf5
 - 1s - loss: 0.2668 - val_loss: 0.2293
Epoch 79/100
Epoch 00079: val_loss did not improve
 - 1s - loss: 0.2670 - val_loss: 0.2558
Epoch 80/100
Epoch 00080: val_loss improved from 0.22930 to 0.22187, saving model to ./hdf5/only_speed_lstm_simple_ChIJsde_glVSUkYRlK8i5BD3BOA.best.hdf5
 - 1s - loss: 0.2663 - val_loss: 0.2219
Epoch 81/100
Epoch 00081: val_loss did not improve
 - 1s - loss: 0.2651 - val_loss: 0.2226
Epoch 82/100
Epoch 00082: val_loss improved from 0.22187 to 0.22015, saving model to ./hdf5/only_speed_lstm_simple_ChIJsde_glVSUkYRlK8i5BD3BOA.best.hdf5
 - 1s - loss: 0.2625 - val_loss: 0.2202
Epoch 83/100
Epoch 00083: val_loss did not improve
 - 1s - loss: 0.2635 - val_loss: 0.2254
Epoch 84/100
Epoch 00084: val_loss did not improve
 - 1s 

Epoch 41/100
Epoch 00041: val_loss did not improve
 - 1s - loss: 0.3197 - val_loss: 0.2799
Epoch 42/100
Epoch 00042: val_loss did not improve
 - 1s - loss: 0.3186 - val_loss: 0.2925
Epoch 43/100
Epoch 00043: val_loss did not improve
 - 1s - loss: 0.3185 - val_loss: 0.2984
Epoch 44/100
Epoch 00044: val_loss did not improve
 - 1s - loss: 0.3144 - val_loss: 0.2795
Epoch 45/100
Epoch 00045: val_loss improved from 0.26521 to 0.25679, saving model to ./hdf5/only_speed_lstm_simple_ChIJozaGTFZSUkYRjY7TpiWcsro.best.hdf5
 - 1s - loss: 0.3149 - val_loss: 0.2568
Epoch 46/100
Epoch 00046: val_loss did not improve
 - 1s - loss: 0.3108 - val_loss: 0.2865
Epoch 47/100
Epoch 00047: val_loss did not improve
 - 1s - loss: 0.3141 - val_loss: 0.2989
Epoch 48/100
Epoch 00048: val_loss did not improve
 - 1s - loss: 0.3092 - val_loss: 0.2722
Epoch 49/100
Epoch 00049: val_loss did not improve
 - 1s - loss: 0.3101 - val_loss: 0.2772
Epoch 50/100
Epoch 00050: val_loss did not improve
 - 1s - loss: 0.3084 - val_l

In [ ]:
multiple_places(place_groups_junctions, 'place_groups_junctions')

Train on 34780 samples, validate on 8696 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.74184, saving model to ./hdf5/only_speed_lstm_simple_ChIJZaR1M1hSUkYRxP0WkwYYy_k.best.hdf5
 - 3s - loss: 0.9266 - val_loss: 0.7418
Epoch 2/100
Epoch 00002: val_loss improved from 0.74184 to 0.62929, saving model to ./hdf5/only_speed_lstm_simple_ChIJZaR1M1hSUkYRxP0WkwYYy_k.best.hdf5
 - 1s - loss: 0.8016 - val_loss: 0.6293
Epoch 3/100
Epoch 00003: val_loss improved from 0.62929 to 0.51400, saving model to ./hdf5/only_speed_lstm_simple_ChIJZaR1M1hSUkYRxP0WkwYYy_k.best.hdf5
 - 1s - loss: 0.6804 - val_loss: 0.5140
Epoch 4/100
Epoch 00004: val_loss improved from 0.51400 to 0.45492, saving model to ./hdf5/only_speed_lstm_simple_ChIJZaR1M1hSUkYRxP0WkwYYy_k.best.hdf5
 - 1s - loss: 0.5620 - val_loss: 0.4549
Epoch 5/100
Epoch 00005: val_loss improved from 0.45492 to 0.40627, saving model to ./hdf5/only_speed_lstm_simple_ChIJZaR1M1hSUkYRxP0WkwYYy_k.best.hdf5
 - 1s - loss: 0.5024 - val_loss: 0.

Epoch 62/100
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.2839 - val_loss: 0.2268
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.2829 - val_loss: 0.2429
Epoch 64/100
Epoch 00064: val_loss improved from 0.22476 to 0.22167, saving model to ./hdf5/only_speed_lstm_simple_ChIJZaR1M1hSUkYRxP0WkwYYy_k.best.hdf5
 - 1s - loss: 0.2800 - val_loss: 0.2217
Epoch 65/100
Epoch 00065: val_loss improved from 0.22167 to 0.22149, saving model to ./hdf5/only_speed_lstm_simple_ChIJZaR1M1hSUkYRxP0WkwYYy_k.best.hdf5
 - 1s - loss: 0.2804 - val_loss: 0.2215
Epoch 66/100
Epoch 00066: val_loss improved from 0.22149 to 0.21662, saving model to ./hdf5/only_speed_lstm_simple_ChIJZaR1M1hSUkYRxP0WkwYYy_k.best.hdf5
 - 1s - loss: 0.2792 - val_loss: 0.2166
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.2760 - val_loss: 0.2300
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.2788 - val_loss: 0.2323
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 1s 